In [7]:
# set up path to import my python scripts
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path+"/src")

In [8]:
# import my functions
from prep_data import get_csv_file_links

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.style.use('ggplot')

In [10]:
# df = pd.read_csv('../data/2020.csv')

## Data Cleaning Notes
The 2020.csv data failed to load because some rows had more than 13 columns.   
Looking at the data I noticed several things.  
- The extra columns were generally '\N' values   
- I did notice one case were a datetime and subsequent rows were in the middle of a previous time and that same datetime was the subsequent row.   
- There was also another issue where it looked like two columns were swapped back and forth. The one was typically 50000 and the other less than 1 so it was easy to see.  

I am going to try and load another year to see if these problem keeps poping up in order to determine how to deal with it.

## IFA readme.txt
```
These are comma seperated files that can easily be imported into statistical software.
Data starts in 1994, recorded as 10 minute averages.
November 2006, switched to 10 second raw numbers.
All data is filtered, out of range and extraneous points have been replaced with "\N".
Fields are:
NAME		UNITS
___________________________________
date		MONTH/DAY/YEAR HH:MM:SS
temperature 	DEGREES CELSIUS
pressure 	MILLIBARS	
humidity 	PERCENT
wind_speed 	METERS/SECOND
wind_direction	DEGREES (CW FROM NORTH)
visibility	METERS
co2		PPM
insolation	LANGLEYS/HOUR
vertical_wind_speed METERS/SECOND
precipitation	INCHES
10min,
dewpoint	DEGREES CELSIUS
```

Weather Conditions of interest

Dew Point Delta
Green - > 10 def F
Yellow - 
Red - < 5 deg F

Humidity
Green < 75
yell < 85
red > 85

Wind
Green - < 20 knots, gusts < 30knots
yellow 
red > 25 knots gusts > 30

Cloud Cover
yellow - anything other than clear down do low elevation ~15deg

fog/precipitation
red for any


### Columns of Interest in determining Green/Yellow/Red Weather

|      | Units | Green | Yellow | Red |
|:-----|------:|------:|-------:|-----|
|date | YYYY-MM-DD HH\:mm\:ss | N/A | N/A | N/A |
|temperature| &deg;C |  N/A | N/A | N/A  |
|humidity|% | <75 | \[75-85) | >85 |
|wind_speed| m/s | sustained < 10 | | > 12 |
|          |     | gusts < 15 | | > 15 |
|visibility| meters | | | |
|precipitation| inches | 0 | | > 0 |
|dewpoint| &deg;C | > 6 | | < 3 |

Need to look at data more to determine map data to threshold for visibility.

In [11]:
column_names = ['date_time','temperature','pressure','humidity','wind_speed','wind_direction','visibility','co2','insolation','vertical_wind_speed','precipitation','10min','dewpoint']
columns_of_interest = ['date_time','temperature','humidity','wind_speed','visibility','precipitation','dewpoint']

In [12]:

base_url = "http://kopiko.ifa.hawaii.edu/weather/archivedata/"
csv_urls = get_csv_file_links(base_url)
print(csv_urls)

['http://kopiko.ifa.hawaii.edu/weather/archivedata/1993.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/1994.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/1995.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/1996.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/1997.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/1998.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/1999.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2000.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2001.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2002.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2003.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2004.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2005.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2006.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2007.csv', 'http://kopiko.ifa.hawaii.edu/weather/archivedata/2008.csv', 'http://kopiko.ifa.hawa

In [13]:
df_2019 = pd.read_csv(csv_urls[-3],na_values='\\N',names=column_names)
df_2018 = pd.read_csv(csv_urls[-4],na_values='\\N',names=column_names)

In [14]:
def get_specific_year(year,url_list):
    if type(year) == int:
        year = str(year)
    elif type(year) != str:
        return 'Year must be a string or integer'
    link = [url for url in url_list if year in url][0]
    return link
    


In [15]:
year = 1994
link = get_specific_year(year,csv_urls)
df_1994 = pd.read_csv(link,na_values='\\N',names=column_names)

In [16]:
year = 1995
link = get_specific_year(year,csv_urls)
df_1995 = pd.read_csv(link,na_values='\\N',names=column_names)

In [17]:
print(df_1994.info())
print(df_1995.info())
print(df_2018.info())
print(df_2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14025 entries, 0 to 14024
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            14025 non-null  object 
 1   temperature          14025 non-null  float64
 2   pressure             14025 non-null  float64
 3   humidity             0 non-null      float64
 4   wind_speed           14025 non-null  float64
 5   wind_direction       14025 non-null  float64
 6   visibility           0 non-null      float64
 7   co2                  0 non-null      float64
 8   insolation           14025 non-null  float64
 9   vertical_wind_speed  14025 non-null  float64
 10  precipitation        14025 non-null  float64
 11  10min                14025 non-null  int64  
 12  dewpoint             0 non-null      float64
dtypes: float64(11), int64(1), object(1)
memory usage: 1.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47400 entries, 0 to 473

In [18]:
df_1994.head()

,date_time,temperature,pressure,humidity,wind_speed,wind_direction,visibility,co2,insolation,vertical_wind_speed,precipitation,10min,dewpoint
0,1994-09-11 17:20:00,13.93,715.0,NaN,7.35,63.77,NaN,NaN,25.83,0.003,0.0,1,NaN
1,1994-09-11 17:30:00,13.48,715.0,NaN,7.29,62.89,NaN,NaN,22.08,0.016,0.0,1,NaN
2,1994-09-11 17:40:00,13.67,715.0,NaN,7.14,62.91,NaN,NaN,18.45,0.015,0.0,1,NaN
3,1994-09-11 17:50:00,13.95,715.0,NaN,7.31,72.30,NaN,NaN,14.95,0.056,0.0,1,NaN
4,1994-09-11 18:00:00,13.85,715.0,NaN,7.96,64.34,NaN,NaN,11.41,0.006,0.0,1,NaN


In [19]:
df_1995.head()

,date_time,temperature,pressure,humidity,wind_speed,wind_direction,visibility,co2,insolation,vertical_wind_speed,precipitation,10min,dewpoint
0,1995-01-01 00:10:00,16.77,714.0,NaN,5.688,337.9,NaN,NaN,65.55,-0.005,0.0,1,NaN
1,1995-01-01 00:20:00,16.46,715.0,NaN,6.897,343.3,NaN,NaN,63.85,-0.086,0.0,1,NaN
2,1995-01-01 00:30:00,16.53,715.0,NaN,7.690,346.2,NaN,NaN,61.92,-0.079,0.0,1,NaN
3,1995-01-01 00:40:00,16.23,714.0,NaN,7.120,344.3,NaN,NaN,60.09,-0.157,0.0,1,NaN
4,1995-01-01 00:50:00,16.14,714.0,NaN,6.119,349.9,NaN,NaN,57.94,-0.193,0.0,1,NaN


In [20]:
df_2018.head()

,date_time,temperature,pressure,humidity,wind_speed,wind_direction,visibility,co2,insolation,vertical_wind_speed,precipitation,10min,dewpoint
0,2018-01-01 00:00:05,12.08,709.12,17.8,5.6,23.0,43682.0,NaN,67.8,-0.1,NaN,0,-42.26
1,2018-01-01 00:00:15,12.09,709.12,18.1,5.3,26.0,43682.0,NaN,67.8,-0.1,NaN,0,-46.06
2,2018-01-01 00:00:25,12.07,709.14,18.3,4.8,35.0,46022.0,NaN,67.8,-0.1,NaN,0,-46.98
3,2018-01-01 00:00:36,12.07,709.16,18.3,3.9,34.0,48408.0,NaN,67.8,-0.1,NaN,0,-38.20
4,2018-01-01 00:00:46,12.06,709.15,18.8,3.6,24.0,48408.0,NaN,67.8,-0.2,NaN,0,-45.39


In [21]:
df_2019.head()

,date_time,temperature,pressure,humidity,wind_speed,wind_direction,visibility,co2,insolation,vertical_wind_speed,precipitation,10min,dewpoint
0,2019-01-01 00:00:01,11.27,707.88,17.0,10.5,120.0,49425.0,NaN,68.9,NaN,NaN,0.0,7.13
1,2019-01-01 00:00:11,11.24,707.93,15.7,9.4,122.0,48363.0,NaN,68.9,NaN,NaN,0.0,4.46
2,2019-01-01 00:00:22,11.26,707.92,15.5,10.2,114.0,48363.0,NaN,68.9,NaN,NaN,0.0,3.08
3,2019-01-01 00:00:32,11.25,707.91,16.2,9.6,123.0,48000.0,NaN,68.9,NaN,NaN,0.0,3.26
4,2019-01-01 00:00:42,11.27,707.91,17.8,9.8,124.0,47351.0,NaN,68.9,NaN,NaN,0.0,5.28


In [22]:
columns_of_interest = ['date_time','temperature','humidity','wind_speed','visibility','precipitation','dewpoint']
print(df_2019[columns_of_interest].head().to_markdown())

|    | date_time           |   temperature |   humidity |   wind_speed |   visibility |   precipitation |   dewpoint |
|---:|:--------------------|--------------:|-----------:|-------------:|-------------:|----------------:|-----------:|
|  0 | 2019-01-01 00:00:01 |         11.27 |       17   |         10.5 |        49425 |             nan |       7.13 |
|  1 | 2019-01-01 00:00:11 |         11.24 |       15.7 |          9.4 |        48363 |             nan |       4.46 |
|  2 | 2019-01-01 00:00:22 |         11.26 |       15.5 |         10.2 |        48363 |             nan |       3.08 |
|  3 | 2019-01-01 00:00:32 |         11.25 |       16.2 |          9.6 |        48000 |             nan |       3.26 |
|  4 | 2019-01-01 00:00:42 |         11.27 |       17.8 |          9.8 |        47351 |             nan |       5.28 |


In [23]:
def count_NaNs(df):
    len_df = len(df)
    max_digits = int(np.log10(len_df) + 1)
    print(f'Total rows          : {len_df}')
    print('-----------------------------')
    print('Number of NaNs per column:')
    for col in df:
        print(f'{col:20}: {sum(df[col].isna()):{max_digits}}')


In [24]:
print(1994)
count_NaNs(df_1994[columns_of_interest])
print('\n1995')
count_NaNs(df_1995[columns_of_interest])
print('\n2018')
count_NaNs(df_2018[columns_of_interest])
print('\n2019')
count_NaNs(df_2019[columns_of_interest])

1994
Total rows          : 14025
-----------------------------
Number of NaNs per column:
date_time           :     0
temperature         :     0
humidity            : 14025
wind_speed          :     0
visibility          : 14025
precipitation       :     0
dewpoint            : 14025

1995
Total rows          : 47400
-----------------------------
Number of NaNs per column:
date_time           :     0
temperature         :     0
humidity            : 47400
wind_speed          :     0
visibility          : 47400
precipitation       :     0
dewpoint            : 47400

2018
Total rows          : 2968957
-----------------------------
Number of NaNs per column:
date_time           :       0
temperature         :    2991
humidity            :    2990
wind_speed          :    2099
visibility          :   12294
precipitation       : 2968957
dewpoint            :  182831

2019
Total rows          : 2422740
-----------------------------
Number of NaNs per column:
date_time           :       0
t